In [179]:
from typing import TypedDict, List, Annotated
import operator
from langgraph.graph import StateGraph, END, START
from langchain_google_genai import ChatGoogleGenerativeAI

In [180]:
class AgentState(TypedDict):
    topic: str
    search_queries : Annotated[List[str], operator.add]
    context: Annotated[List[str], operator.add]
    summary: str
    critique: str
    iterations: Annotated[int, operator.add]


In [204]:
from google import genai
import os

# Initialize the client (ensure GOOGLE_API_KEY is in your env)
# google_client = genai.Client(api_key="************************")
os.environ["GOOGLE_API_KEY"] = "*************************"

In [205]:

from tavily import TavilyClient
client = TavilyClient("tvly-dev-***********************")


In [206]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
)

In [266]:
def generate_queries_node(state: AgentState):
    topic = state["topic"]
    # Prompt the LLM to generate 3 specific search queries
    prompt = f"Give me 3 search queries to research the following topic: {topic}. Return them as comma separated list example output: A,B,C. Make sure there is no comma used other than used to separate search queries."
    
    # Let's say the LLM returns: ["agentic ai architecture", "langgraph vs autogen", "agentic design patterns"]
    new_queries = llm.invoke(prompt) 
    cleaned_queries = new_queries.content.split(",")
    # Return the new list. LangGraph will merge this into the state's 'search_queries'
    return {"search_queries": cleaned_queries}

In [267]:
def research_node(state: AgentState): 
    current_iter = state["iterations"]
    latest_query = state["search_queries"][current_iter]
    cleaned_result = []
    response = client.search(
    query = latest_query, max_results=3
)
#     state["search_queries"] = f"latest info on {topic}"
    for item in response['results']:
        content = item.get('content')
        url = item.get('url')
        cleaned_result.append(f"source ({url}): {content}")
#     formatted_response = "\n--\n".join(cleaned_result)
    return {"context": cleaned_result,
           "iterations": 1}

In [268]:
def writer_node(state: AgentState):
    context = "\n---\n".join(state['context'])
    topic = state['topic']
    response = llm.invoke(f"Please provide a concise summary about {topic} using this info: {context}")
    return {'summary': response.content}
    

In [269]:
def reviewer_node(state: AgentState):
    """Critiques the summary for gaps or errors."""
    prompt = f"Critique this summary about {state['topic']}:\n{state['summary']}\n. If it's perfect, output only 'PASS'. Otherwise, explain what is missing."
    response = llm.invoke(prompt)
    return {"critique": response.content}

In [274]:
def should_continue(state: AgentState):
    if state["iterations"] > 2 or "PASS" == state["critique"]:
        return "end"
    return "continue"

In [275]:
workflow = StateGraph(AgentState)
workflow.add_node("query_gen",generate_queries_node)
workflow.add_node("researcher", research_node)
workflow.add_node("writer", writer_node)
workflow.add_node("reviewer",reviewer_node)

workflow.add_edge(START,"query_gen")
workflow.add_edge("query_gen","researcher")
workflow.add_edge("researcher","writer")
workflow.add_edge("writer","reviewer")

workflow.add_conditional_edges("reviewer",should_continue,{
    "continue": "researcher",
    "end": END
})

# ... add edges and compile
app1 = workflow.compile()

In [276]:
app1.invoke({
    "topic": "future of openai",
})

{'topic': 'future of openai',
 'search_queries': ['OpenAI future AI models and capabilities',
  'OpenAI long-term business strategy and market position',
  'OpenAI vision for artificial general intelligence and societal impact'],
 'context': ['source (https://www.scworld.com/news/openai-outlines-plans-to-prepare-for-future-ai-cybersecurity-capabilities): OpenAI has released plans for a future of AI models with advanced cybersecurity capabilities, outlining plans to prevent misuse and empower',
  'source (https://openai.com/open-models/): * ChatGPT(opens in a new window). * Sora(opens in a new window). # Open models by OpenAI. Advanced open-weight reasoning models to customize for any use case and run anywhere. Download on Hugging Face(opens in a new window)View on GitHub(opens in a new window)Try our models(opens in a new window). ### gpt-oss. Open reasoning models designed to run locally on desktops, laptops, and in data centers—available in 120B and 20B parameters. (opens in a new wi